In [1]:
import joblib
EXP_PATH_NAME="WACA-IF"
joblib.cpu_count()

32

In [2]:
# !pip install --upgrade pip

import os
from joblib import Parallel, delayed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dataclasses
from sklearn.svm import OneClassSVM
from dataclasses import asdict
from tqdm import tqdm
import warnings
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold # Feature selector

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./Classification_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb
RIVAL_CLASSIFIER_TYPE_LST=["OCSVM", "kNN", "LOF"]


np.random.seed(SEED)
print(f"Numpy Seed was set to: {SEED}")

print("Setup Complete")

SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neural Networks utility functions imported--------------------
SEED: 567
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER

In [3]:
import sklearn
sklearn.__dir__()
sklearn.__version__

'1.0.2'

In [4]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
        
    user_ids: list
    num_sample_points_per_exp: int
    exp_begin_cutoff_idx: int
    exp_end_cutoff_idx: int
        
    
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    window_size: int  # After resampling
    ocsvm_step_width: int  # After resampling
    scaler: str  # StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_kernel: str # ocsvm kernel
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects=29,
    max_test_subjects=10,
    user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49],
    num_sample_points_per_exp=21000,
    exp_begin_cutoff_idx=500,
    exp_end_cutoff_idx=-500,
    seconds_per_subject_train=210,
    seconds_per_subject_test=210,    
    window_size=250,
    ocsvm_step_width=250,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_kernel="rbf",
    ocsvm_nu=None,
    ocsvm_gamma=None,
    feature_cols=[
        "x_a",
        "y_a",
        "z_a",
        "x_g",
        "y_g",
        "z_g",
    ],
    exclude_subjects=[],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM_125 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=125
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)


VALID_ROBUST_OCSVM_250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_2000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=2000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

# NORMALIZER_APPROACH (VALID)
# -----------------------------------------------------------
VALID_NORMALIZER_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-NORMALIZER-OCSVM",
    scaler="Normalizer",
    scaler_global=False,
    ocsvm_nu=0.074,
    ocsvm_gamma= 0.029,
)

In [5]:
P = VALID_ROBUST_OCSVM_2000
P.ocsvm_step_width = int(P.window_size * .5)
P.classifier="IF"

P.train_cores=1 # 20 cores for every user and 1 core for the nested crossval function
P.test_cores=2 # 10 cores for every user and 2 for the nested crossval function

# param_dist
param_dist = {
    "model__n_estimators": np.linspace(50, 200, num=10, dtype=np.int64),
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [10],
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [50, 100, 200],
    'model__max_samples':  np.arange(0.1, 1.01, 0.1),
    'model__contamination': np.linspace(0.0001, 0.1, 50),
    'model__max_features': np.arange(0.1, 1.01, 0.1),
    'model__warm_start': [True], 
    }

# param_dist = {
#     "model__n_estimators": [50, 100, 150, 200], #np.linspace(50, 200, num=10, dtype=np.int64),
#     'model__max_samples':  [.5,  .1],
#     'model__contamination': [0.01],
#     'model__max_features': [.5, 1.0],
#     'model__warm_start': [True], 
#     }

# param_dist = {
#     "model__n_estimators": [50, 100], #np.linspace(50, 200, num=10, dtype=np.int64),
#     'model__max_samples':  [.1],
#     'model__contamination': [0.01],
#     'model__max_features': [.5, 1.0],
#     'model__warm_start': [True], 
#     }

In [6]:
param_dist

{'model__n_estimators': [50, 100, 200],
 'model__max_samples': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__contamination': array([0.0001    , 0.00213878, 0.00417755, 0.00621633, 0.0082551 ,
        0.01029388, 0.01233265, 0.01437143, 0.0164102 , 0.01844898,
        0.02048776, 0.02252653, 0.02456531, 0.02660408, 0.02864286,
        0.03068163, 0.03272041, 0.03475918, 0.03679796, 0.03883673,
        0.04087551, 0.04291429, 0.04495306, 0.04699184, 0.04903061,
        0.05106939, 0.05310816, 0.05514694, 0.05718571, 0.05922449,
        0.06126327, 0.06330204, 0.06534082, 0.06737959, 0.06941837,
        0.07145714, 0.07349592, 0.07553469, 0.07757347, 0.07961224,
        0.08165102, 0.0836898 , 0.08572857, 0.08776735, 0.08980612,
        0.0918449 , 0.09388367, 0.09592245, 0.09796122, 0.1       ]),
 'model__max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__warm_start': [True]}

In [7]:
utils_ppp(P)

,Value
name,VALID-ROBUST-OCSVM
frequency,100
max_subjects,29
max_test_subjects,10
user_ids,"[1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 2..."
num_sample_points_per_exp,21000
exp_begin_cutoff_idx,500
exp_end_cutoff_idx,-500
seconds_per_subject_train,210
seconds_per_subject_test,210


In [8]:
utils_eer_scorer

make_scorer(utils_eer, greater_is_better=False)

In [9]:
#include 47 later
# user_ids = [9]
df_exps_dict = load_data_frames(P.user_ids, P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, P.num_sample_points_per_exp)
raw_dfList_exp1, raw_dfList_exp2 = df_exps_dict['dfList_exp1'], df_exps_dict['dfList_exp2']

Loading exp1 data:
1) accel_count: 28388, gyro_count: 31997
2) accel_count: 26010, gyro_count: 28954
3) accel_count: 28227, gyro_count: 31814
4) accel_count: 24860, gyro_count: 26105
5) accel_count: 24270, gyro_count: 24347
6) accel_count: 25012, gyro_count: 25060
7) accel_count: 25301, gyro_count: 25382
8) accel_count: 21975, gyro_count: 21658
19) accel_count: 24110, gyro_count: 25050
21) accel_count: 24326, gyro_count: 23809
22) accel_count: 29123, gyro_count: 28724
26) accel_count: 23148, gyro_count: 24291
27) accel_count: 24299, gyro_count: 23589
28) accel_count: 23807, gyro_count: 24523
29) accel_count: 24030, gyro_count: 23457
35) accel_count: 24388, gyro_count: 23673
36) accel_count: 24228, gyro_count: 24208
37) accel_count: 31945, gyro_count: 31816
38) accel_count: 22135, gyro_count: 22327
39) accel_count: 23573, gyro_count: 23459
40) accel_count: 23057, gyro_count: 24296
41) accel_count: 24102, gyro_count: 23681
42) accel_count: 24074, gyro_count: 24328
43) accel_count: 22631,

In [10]:
randomized_data_idx = list(range(len(P.user_ids)))
random.Random(SEED).shuffle(randomized_data_idx)
split_idx = 2 * (len(randomized_data_idx)//3) + 1
train_set = randomized_data_idx[: split_idx]
test_set = randomized_data_idx[split_idx: ]
# train_set = randomized_data_idx
print(f"train_set: {train_set}\ntest_set: {test_set}")
# train_set = test_set
# test_set = train_set
print(f"train_set: {train_set}\ntest_set: {test_set}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


In [11]:
num_sample_points_per_exp_user_47 = 18000
df_exps_dict_user_47 = load_data_frames([47], P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, num_sample_points_per_exp_user_47)
dfList_exp1_user_47, dfList_exp2_user_47 = df_exps_dict_user_47['dfList_exp1'], df_exps_dict_user_47['dfList_exp2']

raw_dfList_exp1_user_47 = dfList_exp1_user_47
raw_dfList_exp2_user_47 = dfList_exp2_user_47

Loading exp1 data:
47) accel_count: 22777, gyro_count: 22226
Loading exp2 data:
47) accel_count: 17718, gyro_count: 18353


In [27]:
print(f"train_set: {train_set}")
# print(f"X_exp1_train_dic: {X_exp1_train_dic.keys()}")
# print(f"X_exp2_train_dic: {X_exp2_train_dic.keys()}")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5]


In [26]:
print(f"test_set: {test_set}")
# print(f"X_exp1_test_dic: {X_exp1_test_dic.keys()}")
# print(f"X_exp2_test_dic: {X_exp2_test_dic.keys()}")

test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


# 0. No Smoothing
### Optimizing and Testing

In [14]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = None



preprocessing_method=None
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    



EER_df_train_dict={}
EER_df_test_dict={}


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": raw_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": raw_dfList_exp2_user_47
}

test_dict_key=DASH_MACRO_NUM
EER_df_train_dict[test_dict_key] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                param_dist=param_dist)

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\ntest_dict_key: {test_dict_key}\n")
    f.write(EER_df_train_dict[test_dict_key].to_string())

        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  14.130539807956666
MakeWACAXExpDicUnknown Time:  72.46546448627487
Done extracting features



  0%|          | 0/3 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 8683.86it/s]

 33%|███▎      | 1/3 [07:55<15:50, 475.40s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 11433.29it/s]

 67%|██████▋   | 2/3 [15:29<07:42, 462.79s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 12791.41it/s]

100%|██████████| 3/3 [23:32<00:00, 470.67s/it]

  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 12725.44it/s]

 20%|██        | 1/5 [00:09<00:38,  9.63s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 12563.44it/s]

 40%|████      | 2/5 [00:18<00:28,  9.39s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 16396.81it/s]

 60%|██████    | 3/5 [00:28<00:18,  9.49s/it]

run: 3: 100%|██████████| 20/20 [00:00<00:00, 9780.35it/s]

 80%|████████  | 4/5 [00:37<00:09,  9.36s/it]

run: 4: 100%|██████████| 20/20 [00:00<00:00, 15577.73it/s]

 11%|█         | 1/9 [25:47<3:26:21, 1547.71s/it]

window_size: 250
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 167
len_exp2_user_47: 143
MakeWACAXExpDicOwner Time:  6.822429996915162
MakeWACAXExpDicUnknown Time:  37.249191818293184
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10121.39it/s]

 20%|██        | 1/5 [00:03<00:13,  3.34s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 10168.01it/s]

 40%|████      | 2/5 [00:06<00:10,  3.37s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7999.82it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.34s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9318.60it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.34s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8104.94it/s]

 78%|███████▊  | 7/9 [03:35<00:51, 25.56s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7883340520784259
MakeWACAXExpDicUnknown Time:  3.3673316701315343
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10245.00it/s]

 20%|██        | 1/5 [00:03<00:13,  3.35s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8204.82it/s]

 40%|████      | 2/5 [00:06<00:10,  3.37s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8519.81it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.28s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7319.90it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.29s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7271.68it/s]

 89%|████████▉ | 8/9 [03:56<00:24, 24.09s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7661578841507435
MakeWACAXExpDicUnknown Time:  3.1174997189082205
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10031.82it/s]

 20%|██        | 1/5 [00:03<00:13,  3.38s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7708.70it/s]

 40%|████      | 2/5 [00:06<00:10,  3.41s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9181.93it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.40s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8617.84it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.39s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11028.94it/s]

100%|██████████| 9/9 [04:17<00:00, 28.66s/it]


# 1. Butterworth frequency Cut-off

## 1.1 Naive Approach
### Optimizing and Testing

In [15]:
init_experiment_params(exp_config=P)

print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}
    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]
min_key=P.cut_off_freq
print(f"cut_off_freq: {P.cut_off_freq}")

ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


dfList_dict={
            "dfList_exp1": ffted_dfList_exp1,
            "dfList_exp2": ffted_dfList_exp2,
            "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
            "dfList_exp2_user_47": ffted_dfList_exp2_user_47
}
    

# #-------

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"cut_off_freq: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
# #-------
# #-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)


EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
# #-------

reseting experiment params successful!
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 37


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.421264253091067
MakeWACAXExpDicUnknown Time:  32.39162621134892
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 6061.13it/s]

 20%|██        | 1/5 [00:05<00:20,  5.25s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 15893.54it/s]

 40%|████      | 2/5 [00:10<00:15,  5.22s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7638.51it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.22s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8456.26it/s]

 80%|████████  | 4/5 [00:20<00:05,  5.19s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8918.36it/s]

 11%|█         | 1/9 [01:04<08:39, 64.97s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.8801637971773744
MakeWACAXExpDicUnknown Time:  16.524693552870303
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 13135.93it/s]

 20%|██        | 1/5 [00:06<00:26,  6.54s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7341.68it/s]

 40%|████      | 2/5 [00:13<00:20,  6.70s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7891.45it/s]

 60%|██████    | 3/5 [00:20<00:13,  6.68s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 11602.50it/s]

 80%|████████  | 4/5 [00:26<00:06,  6.67s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8093.99it/s]

 78%|███████▊  | 7/9 [04:13<01:04, 32.25s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8235339890234172
MakeWACAXExpDicUnknown Time:  3.4386229729279876
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 12076.89it/s]

 20%|██        | 1/5 [00:04<00:19,  4.95s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 5674.88it/s]

 40%|████      | 2/5 [00:09<00:14,  4.88s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7631.56it/s]

 60%|██████    | 3/5 [00:14<00:09,  4.86s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7927.24it/s]

 80%|████████  | 4/5 [00:19<00:04,  4.81s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8230.58it/s]

 89%|████████▉ | 8/9 [04:42<00:31, 31.16s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7347333841025829
MakeWACAXExpDicUnknown Time:  3.079612380359322
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11990.58it/s]

 20%|██        | 1/5 [00:06<00:25,  6.30s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 16163.02it/s]

 40%|████      | 2/5 [00:12<00:19,  6.35s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7606.64it/s]

 60%|██████    | 3/5 [00:18<00:12,  6.31s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8457.96it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.29s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8128.50it/s]

100%|██████████| 9/9 [05:18<00:00, 35.34s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    
    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 41



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  14.4827835210599
MakeWACAXExpDicUnknown Time:  95.08045976795256
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13637.80it/s]


 33%|███▎      | 1/3 [06:33<13:06, 393.02s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14072.48it/s]


 67%|██████▋   | 2/3 [15:07<07:44, 464.62s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 10873.11it/s]


100%|██████████| 3/3 [23:10<00:00, 463.39s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18633.07it/s]


 20%|██        | 1/5 [00:11<00:47, 11.96s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 10057.08it/s]


 40%|████      | 2/5 [00:23<00:34, 11.48s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 17221.53it/s]


 60%|██████    | 3/5 [00:34<00:22, 11.37s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 10976.98it/s]


 80%|████████  | 4/5 [00:45<00:11, 11.45s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 10290.25it/s]


100%|██████████| 5/5 [00:57<00:00, 11.45s/it]

 22%|██▏       | 2/9 [58:56<3:20:42, 1720.40

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  4.4516099649481475
MakeWACAXExpDicUnknown Time:  25.269359302241355
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 19368.76it/s]


 33%|███▎      | 1/3 [07:11<14:22, 431.12s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15842.51it/s]


 67%|██████▋   | 2/3 [15:16<07:42, 462.79s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14413.42it/s]


100%|██████████| 3/3 [22:25<00:00, 448.39s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 15009.14it/s]


 20%|██        | 1/5 [00:09<00:39,  9.85s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12832.50it/s]


 40%|████      | 2/5 [00:19<00:28,  9.51s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 18067.22it/s]


 60%|██████    | 3/5 [00:28<00:18,  9.36s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 13543.12it/s]


 80%|████████  | 4/5 [00:37<00:09,  9.33s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 15709.00it/s]


100%|██████████| 5/5 [00:46<00:00,  9.36s/it]

 33%|███▎      | 3/9 [1:22:39<2:38:27, 1584.

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  3.5906891161575913
MakeWACAXExpDicUnknown Time:  18.114789451938123
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13171.00it/s]


## 1.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [16]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"



P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]

P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
min_key=P.cut_off_freq
print(f"cut_off_freq: {P.cut_off_freq}")


ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": ffted_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": ffted_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 49


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.222121445927769
MakeWACAXExpDicUnknown Time:  37.77880969597027
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11702.86it/s]

 20%|██        | 1/5 [00:05<00:21,  5.29s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8087.74it/s]

 40%|████      | 2/5 [00:10<00:15,  5.18s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 10212.57it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.30s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7717.21it/s]

 80%|████████  | 4/5 [00:20<00:05,  5.22s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 4611.66it/s]

 11%|█         | 1/9 [01:44<13:59, 104.95s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.012069088872522
MakeWACAXExpDicUnknown Time:  18.851763010956347
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9254.86it/s]

 20%|██        | 1/5 [00:03<00:13,  3.41s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8525.01it/s]

 40%|████      | 2/5 [00:06<00:10,  3.40s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7732.86it/s]

 60%|██████    | 3/5 [00:10<00:07,  3.55s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8050.49it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.50s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10796.15it/s]

 78%|███████▊  | 7/9 [05:15<01:06, 33.37s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8909365660510957
MakeWACAXExpDicUnknown Time:  3.83939236914739
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9894.56it/s]

 20%|██        | 1/5 [00:03<00:13,  3.40s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8607.23it/s]

 40%|████      | 2/5 [00:06<00:10,  3.33s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5874.38it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.36s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8307.20it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.38s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 12310.84it/s]

 89%|████████▉ | 8/9 [05:39<00:30, 30.47s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8270779550075531
MakeWACAXExpDicUnknown Time:  3.604668288026005
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9878.25it/s]

 20%|██        | 1/5 [00:06<00:25,  6.45s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 11169.92it/s]

 40%|████      | 2/5 [00:12<00:19,  6.43s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 15574.84it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.35s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10554.36it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.37s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11462.98it/s]

100%|██████████| 9/9 [06:18<00:00, 42.03s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"

preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())



mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 49



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  12.72940863110125


# 2. Butterworth frequency Cut-off + EMA span
## 2.1 Naive Approach
### Optimizing and Testing

In [17]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"




EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]
P.span=old_test_hyperparameters_df["EMA_span"][0]

min_key= P.cut_off_freq, P.span
print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)

ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)

dfList_dict={
            "dfList_exp1": EMAed_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}
    

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 41, EMA span: 48


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.527643620036542
MakeWACAXExpDicUnknown Time:  32.99298620177433
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 5211.61it/s]

 20%|██        | 1/5 [00:05<00:22,  5.69s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 13870.05it/s]

 40%|████      | 2/5 [00:10<00:15,  5.27s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7936.24it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.13s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8294.06it/s]

 80%|████████  | 4/5 [00:20<00:05,  5.12s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8300.62it/s]

 11%|█         | 1/9 [01:05<08:44, 65.55s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.864102111198008
MakeWACAXExpDicUnknown Time:  16.60937885288149
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11491.24it/s]

 20%|██        | 1/5 [00:03<00:14,  3.64s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7628.78it/s]

 40%|████      | 2/5 [00:07<00:10,  3.50s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8289.14it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.43s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9212.18it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.43s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8280.96it/s]

 78%|███████▊  | 7/9 [03:55<00:53, 26.64s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7892530029639602
MakeWACAXExpDicUnknown Time:  3.1787465531378984
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 14037.16it/s]

 20%|██        | 1/5 [00:03<00:13,  3.32s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8222.51it/s]

 40%|████      | 2/5 [00:06<00:10,  3.35s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9425.40it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.32s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6515.93it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.31s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8203.22it/s]

 89%|████████▉ | 8/9 [04:16<00:24, 24.74s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7394046671688557
MakeWACAXExpDicUnknown Time:  2.8364234040491283
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9248.74it/s]

 20%|██        | 1/5 [00:03<00:13,  3.26s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 13600.21it/s]

 40%|████      | 2/5 [00:06<00:09,  3.29s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8188.80it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.29s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8276.05it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.28s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8444.34it/s]

100%|██████████| 9/9 [04:36<00:00, 30.75s/it]


In [21]:
P.smoothing = "Butter+EMA" 
preprocessing_method="Naive"

rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-kNN/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
P.span=rival_test_hyperparameters_df["EMA_span"][0]

print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")
rival_test_hyperparameters_df

cut_off_freq: 47, EMA span: 41


,window_size,step_width,Mean_EER,best_n_neighbors,cut_off_freq,EMA_span
0,125,62,0.204121,2,47,41
1,250,125,0.168393,4,47,41
2,500,250,0.150881,9,47,41
3,750,375,0.157118,19,47,41
4,1000,500,0.148969,13,47,41
5,1250,625,0.160548,19,47,41
6,1500,750,0.158079,19,47,41
7,1750,875,0.161596,19,47,41
8,2000,1000,0.164788,19,47,41


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"

preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]








EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    key_pair= P.cut_off_freq, P.span, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")
        

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
total cut_off_span_pairs: (2401, 2), choice_num: 13


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 24, EMA span: 30



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.077337847091258
MakeWACAXExpDicUnknown Time:  68.97748926375061
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18375.92it/s]


 33%|███▎      | 1/3 [06:29<12:59, 389.58s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14621.94it/s]


 67%|██████▋   | 2/3 [12:25<06:09, 369.84s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 11572.09it/s]


100%|██████████| 3/3 [17:54<00:00, 358.22s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 28542.39it/s]


 20%|██        | 1/5 [00:06<00:25,  6.31s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15082.00it/s]


 40%|████      | 2/5 [00:12<00:18,  6.17s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 15185.75it/s]


 60%|██████    | 3/5 [00:18<00:12,  6.01s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 13727.06it/s]


 80%|████████  | 4/5 [00:24<00:05,  5.99s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 20743.34it/s]


100%|██████████| 5/5 [00:30<00:00,  6.01s/it]

 67%|██████▋   | 6/9 [1:57:51<56:03, 1121.19

window_size: 1500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 27
len_exp2_user_47: 23
MakeWACAXExpDicOwner Time:  2.4641578728333116
MakeWACAXExpDicUnknown Time:  7.697064138017595
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 24463.72it/s]


 33%|███▎      | 1/3 [05:20<10:41, 320.63s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 16457.93it/s]


 67%|██████▋   | 2/3 [10:30<05:14, 314.05s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 15866.48it/s]


100%|██████████| 3/3 [15:55<00:00, 318.51s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17349.76it/s]


 20%|██        | 1/5 [00:06<00:24,  6.16s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 21631.27it/s]


 40%|████      | 2/5 [00:12<00:18,  6.09s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 12830.54it/s]


 60%|██████    | 3/5 [00:17<00:11,  5.95s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 22086.91it/s]


 80%|████████  | 4/5 [00:23<00:05,  5.91s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14339.50it/s]


100%|██████████| 5/5 [00:29<00:00,  5.93s/it]

 78%|███████▊  | 7/9 [2:14:26<36:00, 1080.17

window_size: 1750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 23
len_exp2_user_47: 19
MakeWACAXExpDicOwner Time:  2.142112142406404
MakeWACAXExpDicUnknown Time:  6.89364912873134
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 11319.13it/s]


 33%|███▎      | 1/3 [05:19<10:38, 319.19s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 16313.90it/s]


 67%|██████▋   | 2/3 [10:25<05:11, 311.74s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 25930.78it/s]


## 2.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [18]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"



P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }

EER_df_test_dict={}


old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]
P.span=old_test_hyperparameters_df["EMA_span"][0]

P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
P.EMA_per_win_span=P.span

min_key= P.cut_off_freq, P.span
print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)

ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)

dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}



EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 40, EMA span: 3


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.514455747324973
MakeWACAXExpDicUnknown Time:  36.624709943309426
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10104.32it/s]

 20%|██        | 1/5 [00:05<00:20,  5.20s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 4375.45it/s]

 40%|████      | 2/5 [00:10<00:15,  5.14s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8196.80it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.09s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8274.42it/s]

 80%|████████  | 4/5 [00:20<00:05,  5.13s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10330.80it/s]

 11%|█         | 1/9 [01:51<14:50, 111.33s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.0140706980600953
MakeWACAXExpDicUnknown Time:  18.83118910714984
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11375.93it/s]

 20%|██        | 1/5 [00:03<00:14,  3.51s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8251.63it/s]

 40%|████      | 2/5 [00:06<00:10,  3.42s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8511.17it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.40s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9950.90it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.39s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8258.13it/s]

 78%|███████▊  | 7/9 [05:34<01:09, 34.62s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7875126679427922
MakeWACAXExpDicUnknown Time:  3.6141937831416726
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9848.10it/s]

 20%|██        | 1/5 [00:06<00:25,  6.42s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9242.63it/s]

 40%|████      | 2/5 [00:12<00:19,  6.37s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 13013.66it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8495.65it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.34s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8013.57it/s]

 89%|████████▉ | 8/9 [06:14<00:36, 36.23s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8085353141650558
MakeWACAXExpDicUnknown Time:  3.3051330568268895
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9738.34it/s]

 20%|██        | 1/5 [00:03<00:14,  3.56s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8994.86it/s]

 40%|████      | 2/5 [00:06<00:10,  3.38s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 14831.34it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.33s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 16057.83it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.36s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8937.36it/s]

100%|██████████| 9/9 [06:38<00:00, 44.24s/it]


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]






P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.EMA_per_win_span=P.span

    key_pair= P.cut_off_freq, P.span, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 3. EMA span
## 3.1 Naive Approach
### Optimizing and Testing

In [19]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"



EER_df_test_dict={}
    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.span=old_test_hyperparameters_df["EMA_span"][0]
min_key=P.span
print(f"EMA span: {P.span}")


EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)

EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)


dfList_dict={
            "dfList_exp1": EMAed_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
EMA span: 37


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.433106846641749
MakeWACAXExpDicUnknown Time:  32.17060551606119
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10325.71it/s]

 20%|██        | 1/5 [00:05<00:21,  5.43s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6987.01it/s]

 40%|████      | 2/5 [00:10<00:16,  5.34s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7866.29it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.22s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8237.05it/s]

 80%|████████  | 4/5 [00:21<00:05,  5.24s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11722.48it/s]

 11%|█         | 1/9 [01:04<08:39, 64.93s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.9368995958939195
MakeWACAXExpDicUnknown Time:  16.74346202192828
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10475.28it/s]

 20%|██        | 1/5 [00:03<00:13,  3.49s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7159.96it/s]

 40%|████      | 2/5 [00:06<00:10,  3.40s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 10852.02it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.40s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9666.52it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.51s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8447.74it/s]

 78%|███████▊  | 7/9 [03:44<00:52, 26.12s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9036291181109846
MakeWACAXExpDicUnknown Time:  3.092913611792028
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 7398.67it/s]

 20%|██        | 1/5 [00:03<00:13,  3.33s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8280.96it/s]

 40%|████      | 2/5 [00:06<00:09,  3.32s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9526.01it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.30s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8908.89it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.35s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9843.47it/s]

 89%|████████▉ | 8/9 [04:05<00:24, 24.45s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8792134267278016
MakeWACAXExpDicUnknown Time:  2.910920205991715
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 12779.72it/s]

 20%|██        | 1/5 [00:06<00:25,  6.31s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8001.34it/s]

 40%|████      | 2/5 [00:12<00:19,  6.44s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8395.32it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 16657.28it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.36s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 14266.34it/s]

100%|██████████| 9/9 [04:41<00:00, 31.32s/it]


In [18]:
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
rival_test_hyperparameters_df
print(rival_test_hyperparameters_file_name)
print(RIVAL_CLASSIFIER_TYPE_LST)

final_experiments_results/WACA-LOF/EMA_Mean_EER_Naive_df_test_dict_raw_df.json
['OCSVM', 'kNN', 'LOF']


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    
    EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

EMA span: 25



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.062254984863102
MakeWACAXExpDicUnknown Time:  70.15507365902886
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18575.31it/s]


 33%|███▎      | 1/3 [06:24<12:49, 384.78s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12274.81it/s]


 67%|██████▋   | 2/3 [12:31<06:14, 374.38s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13586.99it/s]


100%|██████████| 3/3 [19:05<00:00, 381.71s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 18880.50it/s]


 20%|██        | 1/5 [00:06<00:27,  6.75s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 23373.11it/s]


 40%|████      | 2/5 [00:12<00:19,  6.35s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 11642.76it/s]


 60%|██████    | 3/5 [00:19<00:12,  6.38s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 14480.59it/s]


 80%|████████  | 4/5 [00:25<00:06,  6.27s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14011.37it/s]


100%|██████████| 5/5 [00:31<00:00,  6.30s/it]

 22%|██▏       | 2/9 [46:01<2:37:49, 1352.73

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  4.32451828988269
MakeWACAXExpDicUnknown Time:  19.267858140170574
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17795.10it/s]


 33%|███▎      | 1/3 [05:48<11:37, 348.95s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 16269.60it/s]


 67%|██████▋   | 2/3 [11:24<05:40, 340.90s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14055.98it/s]


100%|██████████| 3/3 [17:20<00:00, 346.80s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17288.97it/s]


 20%|██        | 1/5 [00:07<00:29,  7.44s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14844.47it/s]


 40%|████      | 2/5 [00:14<00:21,  7.22s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 14657.71it/s]


 60%|██████    | 3/5 [00:21<00:14,  7.16s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 14883.97it/s]


 80%|████████  | 4/5 [00:28<00:07,  7.13s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 14413.42it/s]


100%|██████████| 5/5 [00:35<00:00,  7.16s/it]

 33%|███▎      | 3/9 [1:04:22<2:03:45, 1237.

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  3.3790314039215446
MakeWACAXExpDicUnknown Time:  13.367395809851587
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 8494.79it/s]


 33%|███▎      | 1/3 [05:35<11:11, 335.68s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 14354.22it/s]


 67%|██████▋   | 2/3 [10:59<05:28, 328.55s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13247.96it/s]


## 3.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [20]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    

EER_df_test_dict={}
    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.span=old_test_hyperparameters_df["EMA_span"][0]
P.EMA_per_win_span=P.span
min_key=P.span
print(f"EMA span: {P.span}")


EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)

EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": EMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
EMA span: 9


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.390540186315775
MakeWACAXExpDicUnknown Time:  37.61850798595697
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 15821.59it/s]

 20%|██        | 1/5 [00:02<00:11,  2.96s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9545.53it/s]

 40%|████      | 2/5 [00:05<00:08,  2.83s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7814.99it/s]

 60%|██████    | 3/5 [00:08<00:05,  2.83s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7897.39it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.89s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 12675.44it/s]

 11%|█         | 1/9 [01:04<08:39, 64.95s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  2.9136509648524225
MakeWACAXExpDicUnknown Time:  18.717424259055406
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9576.04it/s]

 20%|██        | 1/5 [00:03<00:13,  3.50s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7897.39it/s]

 40%|████      | 2/5 [00:06<00:10,  3.39s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9547.70it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.32s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6669.27it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.36s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7018.58it/s]

 78%|███████▊  | 7/9 [03:59<00:54, 27.27s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.8661970007233322
MakeWACAXExpDicUnknown Time:  3.558706567622721
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11497.54it/s]

 20%|██        | 1/5 [00:06<00:25,  6.45s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7977.00it/s]

 40%|████      | 2/5 [00:12<00:19,  6.48s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9600.15it/s]

 60%|██████    | 3/5 [00:19<00:12,  6.39s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8351.86it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.42s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7752.87it/s]

 89%|████████▉ | 8/9 [04:36<00:30, 30.45s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.7439242661930621
MakeWACAXExpDicUnknown Time:  3.1382902581244707
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11871.79it/s]

 20%|██        | 1/5 [00:03<00:13,  3.31s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8109.64it/s]

 40%|████      | 2/5 [00:06<00:09,  3.30s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8920.26it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.31s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 12278.41it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.39s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6583.43it/s]

100%|██████████| 9/9 [04:57<00:00, 33.05s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    P.EMA_per_win_span=P.span

    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                    param_dist=param_dist) 
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
reseting experiment params successful!


  0%|          | 0/3 [00:00<?, ?it/s]

EMA span: 9



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  13.175995624158531
MakeWACAXExpDicUnknown Time:  87.51563867088407
Done extracting features


...


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  12.653837237041444
MakeWACAXExpDicUnknown Time:  89.56300481688231
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17147.60it/s]


 33%|███▎      | 1/3 [07:45<15:31, 465.96s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12580.40it/s]


 67%|██████▋   | 2/3 [15:09<07:32, 452.49s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 18157.16it/s]


100%|██████████| 3/3 [23:02<00:00, 460.82s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17509.10it/s]


 20%|██        | 1/5 [00:09<00:37,  9.34s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 20345.88it/s]


 40%|████      | 2/5 [00:18<00:27,  9.18s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 21361.36it/s]


 60%|██████    | 3/5 [00:27<00:18,  9.31s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 15488.57it/s]


 80%|████████  | 4/5 [00:37<00:09,  9.27s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 21307.11it/s]


100%|██████████| 5/5 [00:46<00:00,  9.24s/it]

 11%|█         | 1/9 [25:46<3:26:12, 1546.58

window_size: 250
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 167
len_exp2_user_47: 143
MakeWACAXExpDicOwner Time:  7.051525607705116
MakeWACAXExpDicUnknown Time:  44.71043225098401
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 13595.80it/s]


 33%|███▎      | 1/3 [06:23<12:47, 383.78s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 15139.16it/s]


# 4. SMA winsize
## 4.1 Naive Approach
### Optimizing and Testing

In [21]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}

old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
min_key=P.winsize
print(f"SMA winsize: {P.winsize}")


SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)

SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)


dfList_dict={
            "dfList_exp1": SMAed_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}
    

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
SMA winsize: 41


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.283681456930935
MakeWACAXExpDicUnknown Time:  32.11145129008219
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10067.94it/s]

 20%|██        | 1/5 [00:05<00:20,  5.07s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 14013.71it/s]

 40%|████      | 2/5 [00:10<00:15,  5.18s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7847.15it/s]

 60%|██████    | 3/5 [00:15<00:10,  5.03s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8802.32it/s]

 80%|████████  | 4/5 [00:20<00:05,  5.05s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8483.62it/s]

 11%|█         | 1/9 [01:04<08:33, 64.24s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.2901418386027217
MakeWACAXExpDicUnknown Time:  16.838665856979787
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8204.82it/s]

 20%|██        | 1/5 [00:03<00:13,  3.39s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7665.03it/s]

 40%|████      | 2/5 [00:06<00:10,  3.40s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8294.06it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.42s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 9425.40it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.40s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7831.04it/s]

 78%|███████▊  | 7/9 [03:46<00:52, 26.22s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9317396688275039
MakeWACAXExpDicUnknown Time:  3.2238987269811332
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8171.25it/s]

 20%|██        | 1/5 [00:03<00:13,  3.29s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 9037.50it/s]

 40%|████      | 2/5 [00:06<00:09,  3.31s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7858.92it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.31s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8336.92it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.35s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11416.18it/s]

 89%|████████▉ | 8/9 [04:07<00:24, 24.61s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9287484749220312
MakeWACAXExpDicUnknown Time:  3.0262923249974847
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11795.01it/s]

 20%|██        | 1/5 [00:06<00:24,  6.22s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7505.91it/s]

 40%|████      | 2/5 [00:12<00:18,  6.21s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8061.32it/s]

 60%|██████    | 3/5 [00:18<00:12,  6.24s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7784.53it/s]

 80%|████████  | 4/5 [00:25<00:06,  6.34s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11078.46it/s]

100%|██████████| 9/9 [04:43<00:00, 31.48s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 4.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [22]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}

old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
P.SMA_per_win_winsize=P.winsize
min_key=P.winsize
print(f"SMA winsize: {P.winsize}")


SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)

SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)


dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}


EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
SMA winsize: 1


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.4173807608895
MakeWACAXExpDicUnknown Time:  35.6612593238242
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11983.73it/s]

 20%|██        | 1/5 [00:02<00:11,  2.92s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8120.63it/s]

 40%|████      | 2/5 [00:05<00:08,  2.94s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8016.64it/s]

 60%|██████    | 3/5 [00:08<00:05,  2.87s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 10854.82it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.91s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 10615.80it/s]

 11%|█         | 1/9 [01:02<08:18, 62.30s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.158256364054978
MakeWACAXExpDicUnknown Time:  19.07352012069896
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 7861.86it/s]

 20%|██        | 1/5 [00:03<00:13,  3.38s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8243.52it/s]

 40%|████      | 2/5 [00:06<00:10,  3.42s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6997.50it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.39s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8859.96it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.38s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11814.94it/s]

 78%|███████▊  | 7/9 [04:03<01:01, 30.63s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.170513005927205
MakeWACAXExpDicUnknown Time:  3.834590128157288
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10894.30it/s]

 20%|██        | 1/5 [00:03<00:13,  3.40s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8756.38it/s]

 40%|████      | 2/5 [00:06<00:10,  3.35s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8789.40it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.35s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5862.88it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.34s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 9519.53it/s]

 89%|████████▉ | 8/9 [04:26<00:28, 28.06s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.049720074981451
MakeWACAXExpDicUnknown Time:  3.49324113689363
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10361.42it/s]

 20%|██        | 1/5 [00:03<00:13,  3.32s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 12557.80it/s]

 40%|████      | 2/5 [00:06<00:09,  3.28s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 9308.26it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.25s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7875.15it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.25s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7084.97it/s]

100%|██████████| 9/9 [04:47<00:00, 31.97s/it]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")

    P.SMA_per_win_winsize=P.winsize


    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 5. Butterworth frequency Cut-off + SMA winsize
## 5.1 Naive Approach
### Optimizing and Testin

In [23]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]

min_key= P.cut_off_freq, P.winsize
print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")


ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)

ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)

dfList_dict={
            "dfList_exp1": SMAed_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}
    

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 43, winsize: 34


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.36455812305212
MakeWACAXExpDicUnknown Time:  31.05014551896602
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 11834.94it/s]

 20%|██        | 1/5 [00:03<00:12,  3.08s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 6651.29it/s]

 40%|████      | 2/5 [00:05<00:08,  2.98s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8736.31it/s]

 60%|██████    | 3/5 [00:08<00:05,  2.92s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7915.27it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.97s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8874.96it/s]

 11%|█         | 1/9 [00:52<06:59, 52.48s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.1472565471194685
MakeWACAXExpDicUnknown Time:  16.993780562188476
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 6945.36it/s]

 20%|██        | 1/5 [00:03<00:14,  3.54s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7384.34it/s]

 40%|████      | 2/5 [00:06<00:10,  3.41s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8048.94it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.38s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 15341.27it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.39s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8407.10it/s]

 78%|███████▊  | 7/9 [03:36<00:52, 26.07s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.1344434707425535
MakeWACAXExpDicUnknown Time:  3.4139310317113996
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8850.61it/s]

 20%|██        | 1/5 [00:03<00:13,  3.44s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 12115.26it/s]

 40%|████      | 2/5 [00:06<00:09,  3.32s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 11096.04it/s]

 60%|██████    | 3/5 [00:09<00:06,  3.31s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8423.99it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.29s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8276.05it/s]

 89%|████████▉ | 8/9 [03:57<00:24, 24.59s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  1.0467480439692736
MakeWACAXExpDicUnknown Time:  3.0281046177260578
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 10509.41it/s]

 20%|██        | 1/5 [00:03<00:13,  3.49s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8624.93it/s]

 40%|████      | 2/5 [00:06<00:09,  3.29s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 6192.68it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.33s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 8269.53it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.35s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7980.03it/s]

100%|██████████| 9/9 [04:18<00:00, 28.73s/it]


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]








EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    key_pair= P.cut_off_freq, P.winsize, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

In [16]:
min_key

(43, 34, 'kNN')

## 5.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [24]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
old_test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/old_{time_of_execution}_{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"


P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

    
old_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
old_test_hyperparameters_df = pd.read_json(old_test_hyperparameters_file_name)
old_test_hyperparameters_df.to_json(f'{old_test_file_name[:-4]}_raw_df.json')

P.winsize=old_test_hyperparameters_df["SMA_winsize"][0]
P.cut_off_freq=old_test_hyperparameters_df["cut_off_freq"][0]

P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
P.SMA_per_win_winsize=P.winsize

min_key= P.cut_off_freq, P.winsize
print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")




ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)

ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)

dfList_dict={
            "dfList_exp1": raw_dfList_exp1,
            "dfList_exp2": SMAed_dfList_exp2,
            "dfList_exp1_user_47": raw_dfList_exp1_user_47,
            "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
}
    
        

EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=old_test_hyperparameters_df)
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
cut_off_freq: 26, winsize: 2


  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  5.584789220243692
MakeWACAXExpDicUnknown Time:  36.41327082179487
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 6475.69it/s]

 20%|██        | 1/5 [00:03<00:12,  3.04s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 7204.23it/s]

 40%|████      | 2/5 [00:05<00:08,  2.90s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5580.50it/s]

 60%|██████    | 3/5 [00:08<00:05,  2.86s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 6594.82it/s]

 80%|████████  | 4/5 [00:11<00:02,  2.92s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 8126.92it/s]

 11%|█         | 1/9 [01:38<13:09, 98.66s/it]

window_size: 250
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  3.075115009676665
MakeWACAXExpDicUnknown Time:  18.526447437237948
Done extracting features


...


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9482.94it/s]

 20%|██        | 1/5 [00:03<00:13,  3.44s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8356.85it/s]

 40%|████      | 2/5 [00:06<00:10,  3.41s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 7606.64it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.38s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7012.71it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.41s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 7734.29it/s]

 78%|███████▊  | 7/9 [05:07<01:06, 33.17s/it]

window_size: 1750
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9749028920195997
MakeWACAXExpDicUnknown Time:  3.7272583567537367
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9578.22it/s]

 20%|██        | 1/5 [00:03<00:13,  3.27s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8658.76it/s]

 40%|████      | 2/5 [00:06<00:09,  3.33s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 8232.20it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 7992.20it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.32s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 13370.43it/s]

 89%|████████▉ | 8/9 [05:31<00:30, 30.42s/it]

window_size: 2000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
MakeWACAXExpDicOwner Time:  0.9563865861855447
MakeWACAXExpDicUnknown Time:  3.379280688241124
Done extracting features



  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 9539.01it/s]

 20%|██        | 1/5 [00:03<00:14,  3.57s/it]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 8261.38it/s]

 40%|████      | 2/5 [00:06<00:10,  3.48s/it]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 10232.51it/s]

 60%|██████    | 3/5 [00:10<00:06,  3.37s/it]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5923.32it/s]

 80%|████████  | 4/5 [00:13<00:03,  3.35s/it]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 11712.66it/s]

100%|██████████| 9/9 [05:55<00:00, 39.55s/it]


In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {CHOICE_NUM_PAIRS}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for clf_type in tqdm(RIVAL_CLASSIFIER_TYPE_LST):
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/WACA-{clf_type}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.SMA_per_win_winsize=P.winsize
    
    key_pair= P.cut_off_freq, P.winsize, clf_type
    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")
    


    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]

EER_df_test_dict[min_key[:-1]] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key[:-1]].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

reseting experiment params successful!
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]
total cut_off_winsize_pairs: (2401, 2), choice_num: 13


  0%|          | 0/3 [00:00<?, ?it/s]

cut_off_freq: 26, winsize: 2



  0%|          | 0/9 [00:00<?, ?it/s]

window_size: 125
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 337
len_exp2_user_47: 289
MakeWACAXExpDicOwner Time:  14.179950281977654
MakeWACAXExpDicUnknown Time:  85.55156582919881
Done extracting features


...



  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 9637.65it/s]


 33%|███▎      | 1/3 [06:23<12:46, 383.21s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 12943.39it/s]


 67%|██████▋   | 2/3 [12:29<06:13, 373.43s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 13831.18it/s]


100%|██████████| 3/3 [18:59<00:00, 379.69s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 12606.87it/s]


 20%|██        | 1/5 [00:06<00:25,  6.33s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 17396.53it/s]


 40%|████      | 2/5 [00:12<00:18,  6.16s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 21913.81it/s]


 60%|██████    | 3/5 [00:18<00:12,  6.11s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 20228.14it/s]


 80%|████████  | 4/5 [00:24<00:06,  6.11s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 20997.77it/s]


100%|██████████| 5/5 [00:30<00:00,  6.09s/it]

 22%|██▏       | 2/9 [47:48<2:43:52, 1404.70s

window_size: 500
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 83
len_exp2_user_47: 71
MakeWACAXExpDicOwner Time:  5.226970039773732
MakeWACAXExpDicUnknown Time:  24.367419144138694
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 20257.45it/s]


 33%|███▎      | 1/3 [05:41<11:23, 341.58s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 10505.46it/s]


 67%|██████▋   | 2/3 [11:09<05:33, 333.54s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 7046.88it/s]


100%|██████████| 3/3 [16:57<00:00, 339.24s/it]


  0%|          | 0/5 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17829.13it/s]


 20%|██        | 1/5 [00:04<00:18,  4.70s/it]


run: 1: 100%|██████████| 20/20 [00:00<00:00, 22429.43it/s]


 40%|████      | 2/5 [00:08<00:13,  4.44s/it]


run: 2: 100%|██████████| 20/20 [00:00<00:00, 20992.51it/s]


 60%|██████    | 3/5 [00:13<00:08,  4.38s/it]


run: 3: 100%|██████████| 20/20 [00:00<00:00, 9937.93it/s]


 80%|████████  | 4/5 [00:17<00:04,  4.35s/it]


run: 4: 100%|██████████| 20/20 [00:00<00:00, 16228.69it/s]


100%|██████████| 5/5 [00:21<00:00,  4.39s/it]

 33%|███▎      | 3/9 [1:06:00<2:06:09, 1261.65

window_size: 750
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19
len(X_exp1_dict_user_47[47]): 55
len_exp2_user_47: 47
MakeWACAXExpDicOwner Time:  4.457716403063387
MakeWACAXExpDicUnknown Time:  17.048368971794844
Done extracting features




  0%|          | 0/3 [00:00<?, ?it/s]


run: 0: 100%|██████████| 20/20 [00:00<00:00, 17600.94it/s]


In [ ]:
1

# 6. The effect of Varying Overlap

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")



time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for overlap in tqdm(OVERLAP_EXP_RANGE):
    overlap*=0.01
    max_window_size=2000
    step_width = int(max_window_size * (1-overlap))
    max_num_windows=max(len(getIndices(sampleSize=max_window_size, step=step_width, numSamplePoints=P.num_sample_points_per_exp)), 100)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    

    EER_df_train_dict[overlap] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                     extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                     param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\overlap: {overlap}\n")
        f.write(EER_df_train_dict[overlap].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["overlap"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# Summary

In [15]:

df=return_and_save_final_result_df_as_json(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST)
df

ValueError: Unexpected character found when decoding 'false'

In [ ]:
relative_df=return_and_save_final_relative_result_df_as_json(df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, 
                                                             window_size_lst=WINDOW_SIZE_LST)
relative_df

In [ ]:
df.style.hide(axis='index').to_latex()


In [ ]:
relative_df.style.hide(axis='index').to_latex()
